In [5]:
import pandas as pd
import numpy as np
import os, glob
filename = os.path.join(os.getcwd(), "FEATURE_ENGINEERING.txt")
df = pd.read_table(filename, low_memory=True)

ImportError: Unable to import required dependencies:
numpy: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.10 from "/Library/Frameworks/Python.framework/Versions/3.10/bin/python3"
  * The NumPy version is: "1.23.1"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: dlopen(/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/_multiarray_umath.cpython-310-darwin.so, 0x0002): tried: '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/_multiarray_u

In [ ]:
df.head(10)

In [ ]:
df.drop(columns=df.columns[0], axis=1, inplace=True)

In [ ]:
df

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split 
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score

In [ ]:
feature_list = list(df.iloc[:, 0:45].select_dtypes(include="float64"))
feature_list

In [ ]:
y = df.loc[:,"arr_ind_10"]
X = df.loc[:,feature_list]

print("Number of examples: " + str(X.shape[0]))
print("\nNumber of Features:" + str(X.shape[1]))
print(str(list(X.columns)))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1234)

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
def train_test_LR(X_train, y_train, X_test, y_test, c=1):
    '''
    Fit a Linear Regression classifier to the training data X_train, y_train.
    Return the loss and accuracy of resulting predictions on the test set.
    Parameters:
        C = Factor that controls how much regularization is applied to the model.
    '''
     # 1. Create the  scikit-learn LogisticRegression model object below and assign to variable 'model'
    model = LogisticRegression(C=c)
  
    # 2. Fit the model to the training data below
    model.fit(X_train, y_train)  
    
    # 3. Make predictions on the test data using the predict_proba() method and assign the result to the 
    # variable 'probability_predictions' below
    probability_predictions = model.predict_proba(X_test)
  
    # 4. Compute the log loss on 'probability_predictions' and save the result to the variable 'l_loss' below
    l_loss = log_loss(y_test, probability_predictions)
        
    # 5. Make predictions on the test data using the predict() method and assign the result to the 
    # variable 'class_label_predictions' below
    class_label_predictions = model.predict(X_test)
        
    # 6. Compute the accuracy score on 'class_label_predictions' and save the result to the variable 'acc_score' below
    acc_score = accuracy_score(y_test, class_label_predictions)
    
    return l_loss, acc_score

In [ ]:
loss, acc = train_test_LR(X_train, y_train, X_test, y_test)
print('Log loss: ' + str(loss))
print('Accuracy: ' + str(acc))

In [ ]:
cs = [10**i for i in range(-10,10)]
cs

In [ ]:
ll_cs = []
acc_cs = []
for i in range(20):
    loss, acc = train_test_LR(X_train, y_train, X_test, y_test, c = cs[i])
    ll_cs.append(loss)
    acc_cs.append(acc)

In [ ]:
plt.figure(figsize=(15,7)) 

ax = sns.barplot(x=cs, y=ll_cs)
g = ax.set_xticklabels([f'10^{i}' for i in range(-10,10)])
ax.set_xlabel('Regularization HyperParameter: C')
ax.set_ylabel('Log Loss')
ax.set_ylim([0.58, 0.605])
g = plt.title('Log Loss Test Performance by Regularization Weight C')

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
x = np.log10(cs)

sns.lineplot(x=x, y=acc_cs, marker='o')

plt.title("Accuracy Test Performance by Regularization Weight C'")
plt.xlabel("Regularization HyperParameter: C")
plt.ylabel("Accuracy")
plt.show()

10000 best hyperparameter

In [ ]:
# 1. Create the  Scikit-learn LogisticRegression model object below and assign to variable 'model_default'
model_default = LogisticRegression(max_iter=1000)

# 2. Fit the model to the training data below
model_default.fit(X_train, y_train)

In [ ]:
# 1. Make predictions on the test data using the predict_proba() method
proba_predictions_default = model_default.predict_proba(X_test)[:,1]

# 2. Make predictions on the test data using the predict() method
class_label_predictions_default = model_default.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_curve

pd.DataFrame(confusion_matrix(y_test, class_label_predictions_default, labels=[True,False]), columns=['Predicted: Arrival 10', 'Predicted: Not Arrival 10'],
index=['Actual: Arrival 10', 'Actual: No Arrival 10'])

In [ ]:
# 1. Create the  model object below and assign to variable 'model_best'
model_best = LogisticRegression(max_iter=1000, C = 10000)

# 2. Fit the model to the training data below
model_best.fit(X_train, y_train)

In [ ]:
# 1. Make predictions on the test data using the predict_proba() method
proba_predictions_best = model_best.predict_proba(X_test)[:,1]

# 2. Make predictions on the test data using the predict() method
class_label_predictions_best = model_best.predict(X_test)

In [ ]:
pd.DataFrame(confusion_matrix(y_test, class_label_predictions_best, labels=[True,False]), columns=['Predicted: Arrival 10', 'Predicted: Not Arrival 10'],
index=['Actual: Arrival 10', 'Actual: No Arrival 10'])

In [ ]:
precision_default, recall_default, thresholds_default = precision_recall_curve(y_test, proba_predictions_default)
precision_best, recall_best, thresholds_best = precision_recall_curve(y_test, proba_predictions_best)

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

In [ ]:
fpr_default, tpr_default, thresholds_default = roc_curve(y_test,proba_predictions_default)
fpr_best, tpr_best, thresholds_best = roc_curve(y_test, proba_predictions_best)

In [ ]:
auc_default = auc(fpr_default, tpr_default)
auc_best = auc(fpr_best, tpr_best)

print(auc_default)
print(auc_best)

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

# Note that k=5 is specifying that we want the top 5 features
selector = SelectKBest(f_classif, k=5)
selector.fit(X, y)
filter = selector.get_support()
top_5_features = X.columns[filter]

print("Best 5 features:")
print(top_5_features)

# Create new training and test data for features
new_X_train = X_train[top_5_features]
new_X_test = X_test[top_5_features]


# Initialize a LogisticRegression model object with the best value of hyperparameter C 
# The model object should be named 'model'
# Note: Supply max_iter=1000 as an argument when creating the model object
model = LogisticRegression(max_iter=1000, C=2.7029857954888498e-05)


# Fit the model to the new training data
model.fit(X_train, y_train)

# Use the predict_proba() method to use your model to make predictions on the new test data 
# Save the values of the second column to a list called 'proba_predictions'
proba_predictions = model.predict_proba(X_test)[:,1]

    
# Compute the auc-roc
fpr, tpr, thresholds = roc_curve(y_test, proba_predictions)
auc_result = auc(fpr, tpr)
print(auc_result)

In [ ]:
feature_list = list(df.iloc[:, 0:55].select_dtypes(include="float64"))
feature_list

In [ ]:
y = df.loc[:,"arr_ind_20"]
X = df.loc[:,feature_list]

print("Number of examples: " + str(X.shape[0]))
print("\nNumber of Features:" + str(X.shape[1]))
print(str(list(X.columns)))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1234)

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
loss, acc = train_test_LR(X_train, y_train, X_test, y_test)
print('Log loss: ' + str(loss))
print('Accuracy: ' + str(acc))

In [ ]:
ll_cs = []
acc_cs = []
for i in range(20):
    loss, acc = train_test_LR(X_train, y_train, X_test, y_test, c = cs[i])
    ll_cs.append(loss)
    acc_cs.append(acc)

In [ ]:
plt.figure(figsize=(15,7)) 

ax = sns.barplot(x=cs, y=ll_cs)
g = ax.set_xticklabels([f'10^{i}' for i in range(-10,10)])
ax.set_xlabel('Regularization HyperParameter: C')
ax.set_ylabel('Log Loss')
ax.set_ylim([0.58, 0.605])
g = plt.title('Log Loss Test Performance by Regularization Weight C')

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
x = np.log10(cs)

sns.lineplot(x=x, y=acc_cs, marker='o')

plt.title("Accuracy Test Performance by Regularization Weight C'")
plt.xlabel("Regularization HyperParameter: C")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
# 1. Create the  Scikit-learn LogisticRegression model object below and assign to variable 'model_default'
model_default = LogisticRegression(max_iter=1000)

# 2. Fit the model to the training data below
model_default.fit(X_train, y_train)

# 1. Make predictions on the test data using the predict_proba() method
proba_predictions_default = model_default.predict_proba(X_test)[:,1]

# 2. Make predictions on the test data using the predict() method
class_label_predictions_default = model_default.predict(X_test)

In [ ]:
pd.DataFrame(confusion_matrix(y_test, class_label_predictions_default, labels=[True,False]), columns=['Predicted: Arrival 20', 'Predicted: Not Arrival 20'],
index=['Actual: Arrival 20', 'Actual: No Arrival 20'])

In [ ]:
from sklearn.svm import l1_min_c

cs = l1_min_c(X_train, y_train, loss="log") * np.logspace(0, 7, 16)
param_grid = dict(C = list(cs))
param_grid

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV

print('Running Grid Search...')

# 1. Create a LogisticRegression model object with the argument max_iter=1000. 
#    Save the model object to the variable 'model'
model = LogisticRegression(max_iter=1000)

# 2. Run a grid search with 5-fold cross-validation and assign the output to the 
# object 'grid'.
grid = GridSearchCV(model, param_grid, cv=5)

# 3. Fit the model on the training data and assign the fitted model to the 
#    variable 'grid_search'
grid_search = grid.fit(X_train, y_train)

print('Done')

In [ ]:
best_c = grid_search.best_params_
best_c

In [ ]:
# 1. Create the  model object below and assign to variable 'model_best'
model_best = LogisticRegression(max_iter=1000, C = 6.106681900802449e-07)

# 2. Fit the model to the training data below
model_best.fit(X_train, y_train)

In [ ]:
# 1. Make predictions on the test data using the predict_proba() method
proba_predictions_best = model_best.predict_proba(X_test)[:,1]

# 2. Make predictions on the test data using the predict() method
class_label_predictions_best = model_best.predict(X_test)

In [ ]:
pd.DataFrame(confusion_matrix(y_test, class_label_predictions_best, labels=[True,False]), columns=['Predicted: Arrival 20', 'Predicted: Not Arrival 20'],
index=['Actual: Arrival 20', 'Actual: No Arrival 20'])

In [ ]:
precision_default, recall_default, thresholds_default = precision_recall_curve(y_test, proba_predictions_default)
precision_best, recall_best, thresholds_best = precision_recall_curve(y_test, proba_predictions_best)

fpr_default, tpr_default, thresholds_default = roc_curve(y_test,proba_predictions_default)
fpr_best, tpr_best, thresholds_best = roc_curve(y_test, proba_predictions_best)

In [ ]:
auc_default = auc(fpr_default, tpr_default)
auc_best = auc(fpr_best, tpr_best)

print(auc_default)
print(auc_best)

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

# Note that k=5 is specifying that we want the top 5 features
selector = SelectKBest(f_classif, k=5)
selector.fit(X, y)
filter = selector.get_support()
top_5_features = X.columns[filter]

print("Best 5 features:")
print(top_5_features)

# Create new training and test data for features
new_X_train = X_train[top_5_features]
new_X_test = X_test[top_5_features]


# Initialize a LogisticRegression model object with the best value of hyperparameter C 
# The model object should be named 'model'
# Note: Supply max_iter=1000 as an argument when creating the model object
model = LogisticRegression(max_iter=1000, C=2.7029857954888498e-05)


# Fit the model to the new training data
model.fit(X_train, y_train)

# Use the predict_proba() method to use your model to make predictions on the new test data 
# Save the values of the second column to a list called 'proba_predictions'
proba_predictions = model.predict_proba(X_test)[:,1]

    
# Compute the auc-roc
fpr, tpr, thresholds = roc_curve(y_test, proba_predictions)
auc_result = auc(fpr, tpr)
print(auc_result)

In [ ]:
# accuracy much higher for arr_ind_20, lower log loss

In [ ]:
feature_list = list(df.iloc[:, 0:36].select_dtypes(include="float64"))
feature_list

In [ ]:
y = df.loc[:,"arr_ind_1"]
X = df.loc[:,feature_list]

print("Number of examples: " + str(X.shape[0]))
print("\nNumber of Features:" + str(X.shape[1]))
print(str(list(X.columns)))

In [ ]:
loss, acc = train_test_LR(X_train, y_train, X_test, y_test)
print('Log loss: ' + str(loss))
print('Accuracy: ' + str(acc))

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
x = np.log10(cs)

sns.lineplot(x=x, y=acc_cs, marker='o')

plt.title("Accuracy Test Performance by Regularization Weight C'")
plt.xlabel("Regularization HyperParameter: C")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
# 1. Create the  Scikit-learn LogisticRegression model object below and assign to variable 'model_default'
model_default = LogisticRegression(max_iter=1000)

# 2. Fit the model to the training data below
model_default.fit(X_train, y_train)

# 1. Make predictions on the test data using the predict_proba() method
proba_predictions_default = model_default.predict_proba(X_test)[:,1]

# 2. Make predictions on the test data using the predict() method
class_label_predictions_default = model_default.predict(X_test)

In [ ]:
# 1. Create the  model object below and assign to variable 'model_best'
model_best = LogisticRegression(max_iter=1000, C = 6.106681900802449e-07)

# 2. Fit the model to the training data below
model_best.fit(X_train, y_train)

In [ ]:
# 1. Make predictions on the test data using the predict_proba() method
proba_predictions_best = model_best.predict_proba(X_test)[:,1]

# 2. Make predictions on the test data using the predict() method
class_label_predictions_best = model_best.predict(X_test)

In [ ]:
pd.DataFrame(confusion_matrix(y_test, class_label_predictions_best, labels=[True,False]), columns=['Predicted: Arrival 1', 'Predicted: Not Arrival 1'],
index=['Actual: Arrival 1', 'Actual: No Arrival 1'])

In [ ]:
precision_default, recall_default, thresholds_default = precision_recall_curve(y_test, proba_predictions_default)
precision_best, recall_best, thresholds_best = precision_recall_curve(y_test, proba_predictions_best)

fpr_default, tpr_default, thresholds_default = roc_curve(y_test,proba_predictions_default)
fpr_best, tpr_best, thresholds_best = roc_curve(y_test, proba_predictions_best)

In [ ]:
auc_default = auc(fpr_default, tpr_default)
auc_best = auc(fpr_best, tpr_best)

print(auc_default)
print(auc_best)

In [ ]:
# Note that k=5 is specifying that we want the top 5 features
selector = SelectKBest(f_classif, k=5)
selector.fit(X, y)
filter = selector.get_support()
top_5_features = X.columns[filter]

print("Best 5 features:")
print(top_5_features)

# Create new training and test data for features
new_X_train = X_train[top_5_features]
new_X_test = X_test[top_5_features]


# Initialize a LogisticRegression model object with the best value of hyperparameter C 
# The model object should be named 'model'
# Note: Supply max_iter=1000 as an argument when creating the model object
model = LogisticRegression(max_iter=1000, C=2.7029857954888498e-05)


# Fit the model to the new training data
model.fit(X_train, y_train)

# Use the predict_proba() method to use your model to make predictions on the new test data 
# Save the values of the second column to a list called 'proba_predictions'
proba_predictions = model.predict_proba(X_test)[:,1]

    
# Compute the auc-roc
fpr, tpr, thresholds = roc_curve(y_test, proba_predictions)
auc_result = auc(fpr, tpr)
print(auc_result)

In [ ]:
feature_list = list(df.iloc[:, 0:65].select_dtypes(include="float64"))
feature_list

In [ ]:
y = df.loc[:,"arr_ind_30"]
X = df.loc[:,feature_list]

print("Number of examples: " + str(X.shape[0]))
print("\nNumber of Features:" + str(X.shape[1]))
print(str(list(X.columns)))

In [ ]:
loss, acc = train_test_LR(X_train, y_train, X_test, y_test)
print('Log loss: ' + str(loss))
print('Accuracy: ' + str(acc))

In [ ]:
# 1. Create the  Scikit-learn LogisticRegression model object below and assign to variable 'model_default'
model_default = LogisticRegression(max_iter=1000)

# 2. Fit the model to the training data below
model_default.fit(X_train, y_train)

# 1. Make predictions on the test data using the predict_proba() method
proba_predictions_default = model_default.predict_proba(X_test)[:,1]

# 2. Make predictions on the test data using the predict() method
class_label_predictions_default = model_default.predict(X_test)

In [ ]:
# 1. Create the  model object below and assign to variable 'model_best'
model_best = LogisticRegression(max_iter=1000, C = 6.106681900802449e-07)

# 2. Fit the model to the training data below
model_best.fit(X_train, y_train)

In [ ]:
# 1. Make predictions on the test data using the predict_proba() method
proba_predictions_best = model_best.predict_proba(X_test)[:,1]

# 2. Make predictions on the test data using the predict() method
class_label_predictions_best = model_best.predict(X_test)

In [ ]:
pd.DataFrame(confusion_matrix(y_test, class_label_predictions_best, labels=[True,False]), columns=['Predicted: Arrival 30', 'Predicted: Not Arrival 30'],
index=['Actual: Arrival 30', 'Actual: No Arrival 30'])

In [ ]:
precision_default, recall_default, thresholds_default = precision_recall_curve(y_test, proba_predictions_default)
precision_best, recall_best, thresholds_best = precision_recall_curve(y_test, proba_predictions_best)

fpr_default, tpr_default, thresholds_default = roc_curve(y_test,proba_predictions_default)
fpr_best, tpr_best, thresholds_best = roc_curve(y_test, proba_predictions_best)

In [ ]:
auc_default = auc(fpr_default, tpr_default)
auc_best = auc(fpr_best, tpr_best)

print(auc_default)
print(auc_best)

In [ ]:
# Note that k=5 is specifying that we want the top 5 features
selector = SelectKBest(f_classif, k=5)
selector.fit(X, y)
filter = selector.get_support()
top_5_features = X.columns[filter]

print("Best 5 features:")
print(top_5_features)

# Create new training and test data for features
new_X_train = X_train[top_5_features]
new_X_test = X_test[top_5_features]


# Initialize a LogisticRegression model object with the best value of hyperparameter C 
# The model object should be named 'model'
# Note: Supply max_iter=1000 as an argument when creating the model object
model = LogisticRegression(max_iter=1000, C=2.7029857954888498e-05)


# Fit the model to the new training data
model.fit(X_train, y_train)

# Use the predict_proba() method to use your model to make predictions on the new test data 
# Save the values of the second column to a list called 'proba_predictions'
proba_predictions = model.predict_proba(X_test)[:,1]

    
# Compute the auc-roc
fpr, tpr, thresholds = roc_curve(y_test, proba_predictions)
auc_result = auc(fpr, tpr)
print(auc_result)